#### Steps

1. Label images in YOLO format using [labellmg](https://tzutalin.github.io/labelImg/)
2. Zip the images and store in a Google Drive folder
3. Run this notebook on Google Colab with GPU
4. Rename `$BACKUP_DIR` accordingly. For example, if the folder is named yolov4_tiny, assign it `/mydrive/yolov4_tiny`
5. Edit `CLASSES` accordingly

In [1]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Mon Jun  7 14:14:49 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import glob
import os
import re
import random

random.seed(42)

In [3]:
CLASSES = ["hold"]

os.environ["NUM_CLASSES"] = f"{len(CLASSES)}"
os.environ["NUM_FILTERS"] = f"{(len(CLASSES) + 5)*3}"
os.environ["CLASS_NAMES"] = "\r\n".join(CLASSES)
os.environ["BACKUP_DIR"] = "/mydrive/yolov4_tiny"

In [4]:
!echo $NUM_CLASSES
!echo $NUM_FILTERS
!echo $CLASS_NAMES
!echo $BACKUP_DIR

1
18
hold
/mydrive/yolov4_tiny


In [5]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/MyDrive/ /mydrive

Mounted at /content/gdrive


**1) Clone the Darknet**



In [6]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15069, done.
remote: Total 15069 (delta 0), reused 0 (delta 0), pack-reused 15069
Receiving objects: 100% (15069/15069), 13.44 MiB | 9.50 MiB/s, done.
Resolving deltas: 100% (10244/10244), done.


**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

**3) Configure Darknet network for training YOLO V4 Tiny**

In [8]:
!cp cfg/yolov4-tiny-custom.cfg cfg/yolov4-tiny-obj.cfg

In [9]:
# filters=(classes + 5)x3 

!sed -i 's/batch=1/batch=64/' cfg/yolov4-tiny-obj.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov4-tiny-obj.cfg
!sed -i 's/max_batches = 500200/max_batches = 8000/' cfg/yolov4-tiny-obj.cfg
!sed -i 's/steps=400000,450000/steps=6400,7200/' cfg/yolov4-tiny-obj.cfg
!sed -i 's/width=416/width=640/' cfg/yolov4-tiny-obj.cfg
!sed -i 's/height=416/height=640/' cfg/yolov4-tiny-obj.cfg
!sed -i "s/random=0/random=1/"
!sed -i "220 s@classes=80@classes=$NUM_CLASSES@" cfg/yolov4-tiny-obj.cfg
!sed -i "269 s@classes=80@classes=$NUM_CLASSES@" cfg/yolov4-tiny-obj.cfg
!sed -i "212 s@filters=255@filters=$NUM_FILTERS@" cfg/yolov4-tiny-obj.cfg
!sed -i "263 s@filters=255@filters=$NUM_FILTERS@" cfg/yolov4-tiny-obj.cfg

!tail -n 20 cfg/yolov4-tiny-obj.cfg

pad=1
filters=18
activation=linear

[yolo]
mask = 0,1,2
anchors = 10,14,  23,27,  37,58,  81,82,  135,169,  344,319
classes=1
num=6
jitter=.3
scale_x_y = 1.05
cls_normalizer=1.0
iou_normalizer=0.07
iou_loss=ciou
ignore_thresh = .7
truth_thresh = 1
random=0
resize=1.5
nms_kind=greedynms
beta_nms=0.6


In [10]:
!echo "$CLASS_NAMES" > data/obj.names
!echo -e "classes = $NUM_CLASSES\r\ntrain  = data/train.txt\r\nvalid  = data/test.txt\r\nnames = data/obj.names\r\nbackup = $BACKUP_DIR" > data/obj.data
!mkdir data/obj
!cat data/obj.data

classes = 1
train  = data/train.txt
valid  = data/test.txt
names = data/obj.names
backup = /mydrive/yolov4_tiny


In [ ]:
# Download weights yolov4 tiny

!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.conv.29 

**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
!rm -r data/obj
!ls data

In [ ]:
!unzip -o $BACKUP_DIR/images.zip -d data/obj

In [ ]:
# We're going to convert the class index on the .txt files. 
# For example, if the indices labelled are 14 and 15, they will be converted to 0 and 1
# Can be doubled checked in `print(index_list)`

index_list = []

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()
        text_converted = []
        print(i, file_path)
        for line in lines:
            numbers = re.findall("[0-9.]+", line)
            if numbers:
                index = int(numbers[0])
                if index not in index_list:
                  index_list.append(index)
                converted_index = index_list.index(index)
                
                text = "{} {} {} {} {}".format(converted_index, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)

        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

print(index_list)

In [ ]:
import glob
images_list = glob.glob("data/obj/*.jpg")
random.shuffle(images_list)

train_images_list = images_list[:4*len(images_list)//5]
test_images_list = images_list[4*len(images_list)//5:]
print(train_images_list)

In [70]:
#Create training.txt and test.txt file

with open("data/train.txt", "w") as f:
  f.write("\n".join(train_images_list))

with open("data/test.txt", "w") as f:
  f.write("\n".join(test_images_list)) 

**5) Start the training**

In [71]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov4-tiny-obj.cfg yolov4-tiny.conv.29 -dont_show

In [77]:
# Validate the weights files to see which performs the best

!./darknet detector map data/obj.data cfg/yolov4-tiny-obj.cfg /mydrive/yolov4_tiny/yolov4-tiny-obj_8000.weights

 CUDA-version: 11000 (11020), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
 0 : compute_capability = 370, cudnn_half = 0, GPU: Tesla K80 
net.optimized_memory = 0 
mini_batch = 1, batch = 16, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 2    640 x 640 x   3 ->  320 x 320 x  32 0.177 BF
   1 conv     64       3 x 3/ 2    320 x 320 x  32 ->  160 x 160 x  64 0.944 BF
   2 conv     64       3 x 3/ 1    160 x 160 x  64 ->  160 x 160 x  64 1.887 BF
   3 route  2 		                       1/2 ->  160 x 160 x  32 
   4 conv     32       3 x 3/ 1    160 x 160 x  32 ->  160 x 160 x  32 0.472 BF
   5 conv     32       3 x 3/ 1    160 x 160 x  32 ->  160 x 160 x  32 0.472 BF
   6 route  5 4 	                           ->  160 x 160 x  64 
   7 conv     64       1 x 1/ 1    160 x 160 x  64 ->  160 x 160 x  64 0.210 BF
   8 route  2 7 	                           ->  1

In [75]:
# Save the config
!cp cfg/yolov4-tiny-obj.cfg /mydrive/yolov4_tiny/yolov4-tiny-obj.cfg